In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("spotify-datalake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
sc = spark.sparkContext


:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lucaslaredo/.ivy2/cache
The jars for the packages stored in: /home/lucaslaredo/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ee58c6bd-9643-48a5-9012-830bd3287ab8;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 442ms :: artifacts dl 19ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3 

In [2]:
# playlist paths
playlists_v1_path = '/shared/sampled/playlists_v1.json'
playlists_v2_path = '/shared/sampled/playlists_v2.json'
playlists_v3_path = '/shared/sampled/playlists_v3.json'

# tracks paths
tracks_v1_path = '/shared/sampled/tracks_v1.json'
tracks_v2_path = '/shared/sampled/tracks_v2.json'
tracks_v3_path = '/shared/sampled/tracks_v3.json'

# Task 1

## Bronze layer

In [5]:
playlists_v1_df = spark.read.json(playlists_v1_path)
tracks_v1_df = spark.read.json(tracks_v1_path)

In [22]:
playlists_v1_df.columns

['collaborative', 'description', 'name', 'pid']

In [23]:
tracks_v1_df.columns

['album_name',
 'album_uri',
 'artist_name',
 'artist_uri',
 'duration_ms',
 'pid',
 'pos',
 'track_name',
 'track_uri']

## Silver layer

In [48]:
from pyspark.sql.functions import col

song_information = tracks_v1_df.select(
    "track_name",
    "track_uri",
    "duration_ms",
    "album_uri",
    "artist_uri"
).distinct()

In [49]:
from pyspark.sql import functions as F

album_information = tracks_v1_df.select(
    "album_name",
    "album_uri",
    "artist_uri"
).distinct()

In [50]:
artists_information = tracks_v1_df.select(
    "artist_name",
    "artist_uri"
).distinct()

In [51]:
playlists_information = playlists_v1_df.select(
    "name",
    "pid",
    "description",
    "collaborative"
).distinct()

In [52]:
playlist_tracks_information = tracks_v1_df.select(
    "pid",
    "pos",
    "track_uri",
    "album_uri",
    "artist_uri"
)

In [126]:
# save all to datalake
import time

# PARQUET
parquet_start_time = time.time()
song_information.write.mode("overwrite").parquet("./silver/parquet/songs/")
album_information.write.mode("overwrite").parquet("./silver/parquet/album_information/")
artists_information.write.mode("overwrite").parquet("./silver/parquet/artists_information/")
playlists_information.write.mode("overwrite").parquet("./silver/parquet/playlists/")
playlist_tracks_information.write.mode("overwrite").parquet("./silver/parquet/playlist_tracks/")
silver_write_parquet_time = time.time() - parquet_start_time

# JSON
json_start_time = time.time()
song_information.write.mode("overwrite").json("./silver/json/songs/")
album_information.write.mode("overwrite").json("./silver/json/album_information/")
artists_information.write.mode("overwrite").json("./silver/json/artists_information/")
playlists_information.write.mode("overwrite").json("./silver/json/playlists/")
playlist_tracks_information.write.mode("overwrite").json("./silver/json/playlist_tracks/")
silver_write_json_time = time.time() - json_start_time

25/02/03 23:30:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/03 23:30:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [54]:
song_information.columns

['track_name', 'track_uri', 'duration_ms', 'album_uri', 'artist_uri']

In [55]:
album_information.columns

['album_name', 'album_uri', 'artist_uri']

In [56]:
artists_information.columns

['artist_name', 'artist_uri']

In [57]:
playlists_information.columns

['name', 'pid', 'description', 'collaborative']

In [58]:
playlist_tracks_information.columns

['pid', 'pos', 'track_uri', 'album_uri', 'artist_uri']

# Gold Layer

In [128]:
read_parquet_start_time = time.time()
silver_playlist_tracks = spark.read.parquet("./silver/parquet/playlist_tracks/")
silver_playlists = spark.read.parquet("./silver/parquet/playlists/")
silver_songs = spark.read.parquet("./silver/parquet/songs/")
silver_artists = spark.read.parquet("./silver/parquet/artists_information/")
silver_album = spark.read.parquet("./silver/parquet/album_information/")
silver_read_parquet_time = time.time() - read_parquet_start_time

read_json_start_time = time.time()
json_silver_playlist_tracks = spark.read.json("./silver/json/playlist_tracks/")
json_silver_playlists = spark.read.json("./silver/json/playlists/")
json_silver_songs = spark.read.json("./silver/json/songs/")
json_silver_artists = spark.read.json("./silver/json/artists_information/")
json_silver_album = spark.read.json("./silver/json/album_information/")
json_silver_read_parquet_time = time.time() - read_json_start_time

In [129]:
silver_songs.columns

['track_name', 'track_uri', 'duration_ms', 'album_uri', 'artist_uri']

In [63]:
silver_playlists.columns

['name', 'pid', 'description', 'collaborative']

In [60]:
silver_playlist_tracks.columns

['pid', 'pos', 'track_uri', 'album_uri', 'artist_uri']

In [84]:
pre_gold_playlist_info.show(5)

[Stage 72:=======>                                                  (1 + 7) / 8]

+-----------+--------------------+--------------------+--------------------+-----------+
|duration_ms|     song_artist_uri|           album_uri|           track_uri|playlist_id|
+-----------+--------------------+--------------------+--------------------+-----------+
|     229120|spotify:artist:1R...|spotify:album:3sY...|spotify:track:004...|     116299|
|     164346|spotify:artist:57...|spotify:album:5K2...|spotify:track:005...|      38723|
|     164346|spotify:artist:57...|spotify:album:5K2...|spotify:track:005...|     151838|
|     273613|spotify:artist:6H...|spotify:album:3oJ...|spotify:track:005...|     109979|
|     273613|spotify:artist:6H...|spotify:album:3oJ...|spotify:track:005...|      18046|
+-----------+--------------------+--------------------+--------------------+-----------+
only showing top 5 rows



In [81]:
from pyspark.sql.functions import sum as spark_sum, countDistinct, col

pre_gold_playlist_info = silver_playlist_tracks.alias("spt").join(
    silver_songs.alias("ss"), col("spt.track_uri") == col("ss.track_uri"), "inner"
).select(
    col("ss.duration_ms"),
    col("ss.artist_uri").alias("song_artist_uri"),  # Renomeia para evitar ambiguidade
    col("ss.album_uri"),
    col("ss.track_uri"),
    col("spt.pid").alias("playlist_id")
)

In [92]:
gold_playlist_info = pre_gold_playlist_info.groupBy("playlist_id").agg(
    col("playlist_id").alias("pid"),
    spark_sum("duration_ms").alias("total_duration_ms"),
    countDistinct("track_uri").alias("num_tracks"),
    countDistinct("song_artist_uri").alias("num_artists"),
    countDistinct("album_uri").alias("num_albums")
).join(silver_playlists, "pid", "inner").select(
    col("pid").alias("playlist_id"),
    "total_duration_ms",
    "num_tracks",
    "num_artists",
    "num_albums",
    "name",
    "description"
)

In [93]:
gold_playlist_info.show(5)

25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:10:36 WARN RowBasedKeyValueBatch: Calling spill() on

+-----------+-----------------+----------+-----------+----------+---------------+-----------+
|playlist_id|total_duration_ms|num_tracks|num_artists|num_albums|           name|description|
+-----------+-----------------+----------+-----------+----------+---------------+-----------+
|      11276|         11314441|        51|         17|        25|   Stuff I Like|       null|
|      13460|          3221562|        15|          4|         8|     21 Savage |       null|
|      37884|          8341215|        18|          8|         9|           Mood|       null|
|     105182|          1202078|         5|          5|         5|     fave songs|       null|
|     116139|          9364728|        38|         28|        31|This is for you|       null|
+-----------+-----------------+----------+-----------+----------+---------------+-----------+
only showing top 5 rows



In [102]:
silver_artists.columns

['artist_name', 'artist_uri']

In [107]:
gold_playlist_tracks_v1 = silver_playlist_tracks.alias("spt").join(
    silver_songs.alias("ss"), col("spt.track_uri") == col("ss.track_uri"), "inner"
).select(
    col("ss.artist_uri"),
    col("ss.album_uri"),
    col("ss.track_name"),
    col("spt.pos"),
    col("spt.pid").alias("playlist_id")
)

In [108]:
gold_playlist_tracks_v1.show(5)

[Stage 144:==============>                                          (2 + 6) / 8]

+--------------------+--------------------+--------------------+---+-----------+
|          artist_uri|           album_uri|          track_name|pos|playlist_id|
+--------------------+--------------------+--------------------+---+-----------+
|spotify:artist:1R...|spotify:album:3sY...|            Magnolia| 12|     116299|
|spotify:artist:57...|spotify:album:5K2...|I Hear A Symphony...|  5|      38723|
|spotify:artist:57...|spotify:album:5K2...|I Hear A Symphony...| 27|     151838|
|spotify:artist:6H...|spotify:album:3oJ...|     Fuck Everything|  9|     109979|
|spotify:artist:6H...|spotify:album:3oJ...|     Fuck Everything| 22|      18046|
+--------------------+--------------------+--------------------+---+-----------+
only showing top 5 rows



In [121]:
gold_playlist_tracks = gold_playlist_tracks_v1.join(
    silver_artists, "artist_uri", "inner"
).select(
    "playlist_id",
    "pos",
    "track_name",
    "album_uri",
    "artist_name"
).join(
    silver_album, "album_uri", "inner"
).select(
    "playlist_id",
    "pos",
    "track_name",
    "artist_name",
    "album_name"
)

In [122]:
gold_playlist_tracks.show(5)

+-----------+---+--------------------+---------------+--------------------+
|playlist_id|pos|          track_name|    artist_name|          album_name|
+-----------+---+--------------------+---------------+--------------------+
|     116299| 12|            Magnolia| The Hush Sound|          Like Vines|
|      38723|  5|I Hear A Symphony...|   The Supremes|I Hear A Symphony...|
|     151838| 27|I Hear A Symphony...|   The Supremes|I Hear A Symphony...|
|     109979|  9|     Fuck Everything|Suicide Silence|     The Black Crown|
|      18046| 22|     Fuck Everything|Suicide Silence|     The Black Crown|
+-----------+---+--------------------+---------------+--------------------+
only showing top 5 rows



In [123]:
# saving gold layer

gold_playlist_info.write.mode("overwrite").parquet("./gold/playlists/")
gold_playlist_tracks.write.mode("overwrite").parquet("./gold/playlist_tracks/")

25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/03 23:23:22 WARN RowBasedKeyValueBatch: Calling spill() on

In [124]:
import time

# JSON Load
start_time = time.time()
df_json = spark.read.json("./gold/playlist_info_json")
json_time = time.time() - start_time

# Parquet Load
start_time = time.time()
df_parquet = spark.read.parquet("/datalake/gold/playlist_info_parquet")
parquet_time = time.time() - start_time

print(f"JSON Load Time: {json_time:.2f} seconds")
print(f"Parquet Load Time: {parquet_time:.2f} seconds")


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/datalake/gold/playlist_info_json.